In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error

#retrieve
test_file_path = '/Users/mgupta/Downloads/all/test2.csv'
train_file_path = '/Users/mgupta/Downloads/all/train.csv'
test = pd.read_csv(test_file_path)
train = pd.read_csv(train_file_path)
gender_submission_file_path = '/Users/mgupta/Downloads/all/gender_submission.csv'
gender_submission = pd.read_csv(gender_submission_file_path)

#clean gender(male = 0 and female = 1)
train['Sex'] = train['Sex'].replace(['male'],0)
train['Sex'] = train['Sex'].replace(['female'],1)
test['Sex'] = test['Sex'].replace(['male'],0)
test['Sex'] = test['Sex'].replace(['female'],1)

#clean embarked
train['Embarked'] = train['Embarked'].replace(['C'],1)
train['Embarked'] = train['Embarked'].replace(['Q'],2)
train['Embarked'] = train['Embarked'].replace(['S'],3)
test['Embarked'] = test['Embarked'].replace(['C'],1)
test['Embarked'] = test['Embarked'].replace(['Q'],2)
test['Embarked'] = test['Embarked'].replace(['S'],3)
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].median())
test['Embarked'] = test['Embarked'].fillna(test['Embarked'].median())


In [2]:
train_age_null = train.Age.isnull()
train_age_null_df = train[train_age_null]
test_age_null = test.Age.isnull()
test_age_null_df = test[test_age_null]

In [3]:
#clean age by gender and impute with mean by gender
def clean_age(filename,df_name):
    ret_lst = []
    df = filename.Sex == 0
    male_mean = filename[df].Age.mean()
    df_f = filename.Sex == 1
    female_mean = filename[df_f].Age.mean()
    for index, column in df_name.iterrows():
        if column.Sex == 0:
            ret_lst.append(male_mean)
        else:
            ret_lst.append(female_mean)
    print(female_mean)
    print(male_mean)
    return ret_lst

In [4]:
lst_age_train = clean_age(train,train_age_null_df)
lst_age_test = clean_age(test,test_age_null_df)

27.915708812260537
30.72664459161148
30.27236220472441
30.27273170731707


In [5]:
train_age_null_df.drop('Age',axis = 1)
test_age_null_df.drop('Age',axis = 1)
train_age_null_df['Age'] = lst_age_train
test_age_null_df['Age'] = lst_age_test

/Users/mgupta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mgupta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [6]:
train[train.Age.isnull()] = train_age_null_df
test[test.Age.isnull()] = test_age_null_df

In [7]:
#establish role column
def rol_col(filename):
    ret = []
    for name in filename['Name']:
        if "Mme" in name:
            ret.append(1)
        elif 'Mr' in name:
            ret.append(2)
        elif 'Mlle' or 'Miss' or 'Ms' in name:
            ret.append(3)
        elif 'Lady' or 'Countess' or 'Dona' in name:
            ret.append(4)
        elif 'Capt' or 'Col' or 'Major' or 'Rev' in name:
            ret.append(5)
        elif 'Jonkheer' or 'Don' or 'Sir' in name:
            ret.append(6)
        elif ('Dr' in name) and filename['Sex'] == 'male':
            ret.append(2)
        elif ('Dr' in name) and filename['Sex'] == 'female':
            ret.append(1)
    filename['Role'] = ret
    filename['Role'] = filename['Role'].fillna(0)

In [8]:
#establish alone column
def alone_col(filename):
    ret = []
    for num in filename['SibSp']:
        if num == 0:
            ret.append(2)
        else:
            ret.append(1)
    index = 0
    for num in filename['Parch']:
        if num == 0 and ret[index] == 2:
            ret[index] = 2
        else:
            ret[index] = 1
        index += 1
    filename['Alone'] = ret
    filename['Alone'] = filename['Alone'].fillna(0)

In [9]:
#apply method to training and test sets
rol_col(test)
rol_col(train)
alone_col(train)
alone_col(test)

In [10]:
#establish training and target columns
columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Fare','Embarked','Role','Parch','Alone']
y = train.Survived
X = train[columns]

In [11]:
#determine optimal max_depth for decision tree(47 is optimal)
for num in range(1,50):
    training_model = RandomForestClassifier(max_depth = num)
    training_model.fit(X,y)
    test = test[columns].fillna(test.mean()).copy()
    training_predicts = training_model.predict(X)
    print(str(accuracy_score(y, training_predicts)))

0.7822671156004489
0.7833894500561167
0.8226711560044894
0.835016835016835
0.8473625140291807
0.8597081930415263
0.8911335578002245
0.9147025813692481
0.9135802469135802
0.9326599326599326
0.9483726150392817
0.9629629629629629
0.9618406285072951
0.9618406285072951
0.9607182940516273
0.9607182940516273
0.9719416386083053
0.9753086419753086
0.9640852974186308
0.9797979797979798
0.9696969696969697
0.9629629629629629
0.9663299663299664
0.9719416386083053
0.9708193041526375
0.978675645342312
0.9663299663299664
0.9696969696969697
0.9753086419753086
0.9719416386083053
0.9696969696969697
0.9663299663299664
0.9640852974186308
0.9753086419753086
0.9719416386083053
0.9708193041526375
0.9741863075196409
0.9764309764309764
0.9708193041526375
0.9685746352413019
0.9730639730639731
0.9652076318742986
0.9618406285072951
0.9663299663299664
0.9719416386083053
0.9685746352413019
0.9719416386083053
0.9674523007856342
0.9708193041526375


In [12]:
#determine optimal n_estimators(100 is optimal)
for num in [10,20,30,50,100]:
    training_model = RandomForestClassifier(n_estimators = num)
    training_model.fit(X,y)

    test = test[columns].fillna(test.mean()).copy()
    training_predicts = training_model.predict(X)
    print(accuracy_score(y, training_predicts))

0.9741863075196409
0.9764309764309764
0.9820426487093153
0.9842873176206509
0.9854096520763187


In [13]:
#split into train-test
train_X, val_X, train_y, val_y = train_test_split(X,y,random_state = 42)
my_model = RandomForestClassifier(n_estimators = 100, max_depth = 47, random_state = 42)
my_model.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=47, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [14]:
#predict for test and view
predictions = my_model.predict(test[columns])
predictions

array([0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [259]:
#prepare submission file
submit = pd.DataFrame({
    'PassengerId': gender_submission['PassengerId'],
    'Survived': predictions
})

submit.to_csv('submission7.csv', index = False)
